# Reading data from EPT

## Introduction

This tutorial describes how to use [Conda], [Entwine], [PDAL], and [GDAL] to
read data from the [USGS 3DEP AWS Public Dataset]. We will be using PDAL's
[readers.ept] to fetch data, we will filter it for noise using [filters.outlier],
we will classify the data as ground/not-ground using [filters.smrf], and we will
write out a digital terrain model with {ref}`writers.gdal`. Once our elevation model
is constructed, we will use GDAL [gdaldem] operations to create hillshade, slope,
and color relief.

## Write the Pipeline

PDAL uses the concept of [pipelines] to describe the reading, filtering, and writing
of point cloud data. We will construct a pipeline that will do a number of things
in succession.

:::{figure} images/pipeline-example-overview.png
:scale: 75%

Pipeline diagram. The data are read from the [Entwine Point Tile] resource at
<https://usgs.entwine.io> for Iowa using {ref}`readers.ept` and filtered through a
number of steps until processing is complete. The data are then written to
an `iowa.laz` and `iowa.tif` file.
:::

In [ ]:
import os
import sys

conda_env_path = os.environ.get('CONDA_PREFIX', sys.prefix)
proj_data = os.path.join(os.path.join(conda_env_path, 'share'), 'proj')
os.environ["PROJ_DATA"] = proj_data

Our first step is to import the `pdal` library.

In [ ]:
import pdal

Nearly all PDAL pipelines will naturally start with a reader stage to load point cloud data from a source. In this case, we will define an EPT reader where the filename is URL pointing to the `ept.json` for this particular dataset. To reduce the amount of data transfered, we will define a `bounds` to spatially subset the data while reading.

In [ ]:
pipeline = pdal.Reader.ept(
    "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json",
    bounds="([-10425171.940, -10423171.940], [5164494.710, 5166494.710])"
)

In the following steps, we will continue to build out the pipeline by appending filter stages, starting with an expression filter that passes only those points that are **not** marked with the ASPRS classification label of 7 indicating noise.

In [ ]:
pipeline |= pdal.Filter.expression(expression="Classification != 7")

:::{note}
Formerly, this step may have appeared as `pdal.Filter.range(limits="Classification![7:7]")`. While this syntax is still supported, many users will find the more natural expressions supported in the expression filter easier to write and interpret.
:::

Next, the classification labels of all remaining points are reset to zero using the assignment filter.

In [ ]:
pipeline |= pdal.Filter.assign(assignment="Classification[:]=0")

Because the EPT data is in web mercator, we will reproject to UTM zone 15 prior to any further processing.

In [ ]:
pipeline |= pdal.Filter.reprojection(out_srs="EPSG:26915")

With coordinates now in rectilinear coordinates and all labels reset to zero, we apply the SMRF filter to label all points as either ground or unclassified.

In [ ]:
pipeline |= pdal.Filter.smrf()

Following this segmentation step, we can apply another expression (or range) filter to only retain those points classified as ground (ASPRS label 2).

In [ ]:
pipeline |= pdal.Filter.expression(expression="Classification == 2")

Following this step, we will execute the pipeline and report the number of points in the resulting point cloud.

In [ ]:
pipeline.execute()
print(f"Processed point cloud contains {len(pipeline.arrays[0])} points")